In [1]:
!nvidia-smi

Sun Sep 14 02:33:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P0             27W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# `Importing Libraries and Loading Data`

---



In [2]:
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == 

In [3]:
from datasets import load_dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, DataCollatorWithPadding
import evaluate

from huggingface_hub import login
from google.colab import userdata
import wandb

2025-09-14 02:33:47.602947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757817227.744260      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757817227.790932      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
#getting the secret keys
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
hf_w_token= user_secrets.get_secret("HF_W_TOKEN")
wandb_key= user_secrets.get_secret("wandb_key")


In [5]:
#Logging into huggingface
login(token=hf_token)

In [6]:
#checking device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'The device is: {device}')

The device is: cuda


In [7]:
#Calling model and tokenizer
model_chkpt = "google/bigbird-roberta-base"
dataset_name = "Eladio/emrqa-msquad"
new_name = f"{model_chkpt[7:]}-finetuned-{dataset_name[7:]}"

tokenizer = AutoTokenizer.from_pretrained(model_chkpt)
model = AutoModelForQuestionAnswering.from_pretrained(model_chkpt).to(device)

tokenizer_config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/846k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

Some weights of BigBirdForQuestionAnswering were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['qa_classifier.intermediate.dense.bias', 'qa_classifier.intermediate.dense.weight', 'qa_classifier.output.LayerNorm.bias', 'qa_classifier.output.LayerNorm.weight', 'qa_classifier.output.dense.bias', 'qa_classifier.output.dense.weight', 'qa_classifier.qa_outputs.bias', 'qa_classifier.qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [8]:
new_name

'bigbird-roberta-base-finetuned-emrqa-msquad'

In [9]:
#loading dataset and subseting to about half
ds = load_dataset(dataset_name)
ds["train"] = ds["train"].select(range(65000))
ds["validation"] = ds["validation"].select(range(10000))

README.md:   0%|          | 0.00/670 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/43.9M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/11.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130956 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/32739 [00:00<?, ? examples/s]

In [10]:
ds

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answers'],
        num_rows: 65000
    })
    validation: Dataset({
        features: ['context', 'question', 'answers'],
        num_rows: 10000
    })
})

In [11]:
train_ds = ds["train"]
train_ds[0]

{'context': "The patient was admitted on 5/5/2006 with a history of mechanical fall, with the attending physician being Dr. Clemente Armand Bolstad, with a full code status and disposition of Rehabilitation. Medications on Admission included Amiodarone 100 QD, Colace 100 bid, lasix 40mg QD, Glyburide 5mg bid, Plaquenil 200mg bid, Isordil 20mg tid, Lisinopril 20mg QD, Coumadin 5mg 3dys/week, 2.5mg 4dys/week, Norvasc 10mg QD, Neurontin 300mg TID, with APAP prn. An override was added on 10/2/06 by Gerad E. Dancy, PA for POTENTIALLY SERIOUS INTERACTION: AMIODARONE HCL & WARFARIN with the reason for override being monitoring. The patient was rehydrated with IVF and PO's were encouraged, holding Glypizide while in house, Novolog sliding scale was started on 1/2, Low dose NPH 6 units BID was started on 1/2, bridged with lovenox and INR therapeutic 1/2 and restarted on home regimen of 5/2.5mg variable dose. Pain was controlled with TYLENOL (ACETAMINOPHEN) 650 MG PO Q4H PRN Pain, Headache. A CT

In [12]:
train_ds.features

{'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': {'answer_end': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
  'answer_start': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
  'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}}

# `Preprocessing the Data`

In [13]:
def get_token_idx(batch,tokenized):
  """
  function to get start tokens and end tokens indices

  Args:
    batch: batch of rows
    tokenized: tokenized rows

  Returns:
    start and end tokens indices
  """
  start_positions = []
  end_positions = []

  for i in range(len(batch["question"])):
    char_start = batch["answers"][i]["answer_start"][0]
    char_end = batch["answers"][i]["answer_end"][0]
    token_start = None
    token_end = None

    # Iterate through the offset mapping for the current example
    for j, (start, end) in enumerate(tokenized["offset_mapping"][i]):
      if start <= char_start < end:
        token_start = j
      if start < char_end <= end:
        token_end = j
        break

    start_positions.append(token_start)
    end_positions.append(token_end)

  return start_positions,end_positions



def preprocess(batch,tokenizer,max_length=4096):
  """
  Preprocess medical QA data for extractive question answering.

  Args:
      Batch: Batch of rows with 'question', 'context', 'answer' fields
      tokenizer: Hugging Face tokenizer
      max_length: Maximum sequence length

  Returns:
      Dictionary with tokenized inputs and token positions
  """
  questions = batch["question"]
  contexts = batch["context"]

  tokenized = tokenizer(
      questions,
      contexts,
      truncation=True,
      padding=False,
      max_length=max_length,
      return_tensors=None,
      return_offsets_mapping = True
  )


  start_positions,end_positions = get_token_idx(batch,tokenized)

  tokenized["start_positions"] = start_positions
  tokenized["end_positions"] = end_positions

  tokenized.pop("offset_mapping")

  return tokenized

In [14]:
#tokenizing and getting start and end positions
preprocessed_data = ds.map(lambda batch:preprocess(batch,tokenizer),batched=True,remove_columns=["question", "context", "answers"])

Map:   0%|          | 0/65000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [15]:
preprocessed_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 65000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10000
    })
})

# ` Setting up Trainer`

In [16]:
ds["validation"][1]["answers"]["answer_start"][0]

356

In [17]:
squad_metric = evaluate.load("squad")

def compute_metrics(eval_preds):
    """
    Computing F1 Score and EM metrics
    """

    start_logits,end_logits = eval_preds.predictions

    predictions = []
    references = []

    for i in range(len(start_logits)):
        # get start and end positions
        start_i = start_logits[i].argmax()
        end_i = end_logits[i].argmax()+1

        input_ids = preprocessed_data["validation"][i]["input_ids"]
        pred_text = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[start_i:end_i])
        )

        ref_text = ds["validation"][i]["answers"]["text"][0]

        predictions.append({
            "id":str(i),
            "prediction_text":pred_text
        })

        references.append({
            "id":str(i),
            "answers":{
                "text":[ref_text],
                "answer_start":[ds["validation"][i]["answers"]["answer_start"][0]]
            }
        })

    #compute metrics
    results = squad_metric.compute(predictions=predictions,references=references)
    return {
        "exact_match": results["exact_match"],
        "f1": results["f1"]
    }

In [18]:
#setting up data collator for dynamic padding
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    return_tensors="pt",
    padding=True
)

#batch and logging steps
batch_size = 4
logging_steps = len(preprocessed_data["train"])//batch_size

#setting up wandb
wandb.login(key=wandb_key)
wandb.init(
    project=new_name,
    name="big bird roberta-run-2",
    config={
        "model": "Big-Bird-Roberta-Base",
        "dataset": "emrqa-msquad",
        "epochs": 3,
        "batch_size": 4,
        "gradient_accumulation_steps":16,
        "learning_rate":2e-4,
        "fp16":True,
        "save_strategy":"steps",
        "eval_strategy":"steps",
        "eval_steps":400,
        "save_steps":400,
        "warmup_steps":50
    }
)

#training Arguements
training_args = TrainingArguments(
    output_dir=new_name,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size*2,
    gradient_accumulation_steps=16,
    weight_decay=0.01,
    logging_steps=logging_steps,
    log_level="info",
    disable_tqdm=False,
    report_to="wandb",
    run_name="big-bird-roberta-run-2",
    save_strategy="steps",
    eval_strategy="steps",
    eval_steps=400,
    save_steps=400,
    warmup_steps=50,
    load_best_model_at_end=True
)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: diaa_k (diaa_k-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250914_023516-fy13daha
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run big bird roberta-run-2
wandb: ⭐️ View project at https://wandb.ai/diaa_k-personal/bigbird-roberta-base-finetuned-emrqa-msquad
wandb: 🚀 View run at https://wandb.ai/diaa_k-personal/bigbird-roberta-base-finetuned-emrqa-msquad/runs/fy13daha


In [19]:
# Make all parameters contiguous in-place
for name, param in model.named_parameters():
    if not param.data.is_contiguous():
        print(f"Making contiguous: {name}")
        param.data = param.data.contiguous()


# `Training and Evaluating`

In [20]:
#training the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=preprocessed_data["train"],
    eval_dataset=preprocessed_data["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipykernel_19/2020438904.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Using auto half precision backend
***** Running training *****
  Num examples = 65,000
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 16
  Total optimization steps = 3,048
  Number of trainable parameters = 131,603,714
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Attention type 'block_sparse' is not possible if sequence_length: 699 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_

Step,Training Loss,Validation Loss,Exact Match,F1
400,No log,200.938553,61.780000,79.781293
800,No log,200.406921,81.980000,90.124934
1200,No log,200.226486,90.380000,94.864451
1600,No log,200.168106,92.930000,96.299659
2000,No log,200.107483,95.590000,97.740372
2400,No log,200.063904,97.080000,98.631928
2800,No log,200.044708,97.910000,98.992393



***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8
Saving model checkpoint to bigbird-roberta-base-finetuned-emrqa-msquad/checkpoint-400
Configuration saved in bigbird-roberta-base-finetuned-emrqa-msquad/checkpoint-400/config.json
Model weights saved in bigbird-roberta-base-finetuned-emrqa-msquad/checkpoint-400/model.safetensors
tokenizer config file saved in bigbird-roberta-base-finetuned-emrqa-msquad/checkpoint-400/tokenizer_config.json
Special tokens file saved in bigbird-roberta-base-finetuned-emrqa-msquad/checkpoint-400/special_tokens_map.json

***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8
Saving model checkpoint to bigbird-roberta-base-finetuned-emrqa-msquad/checkpoint-800
Configuration saved in bigbird-roberta-base-finetuned-emrqa-msquad/checkpoint-800/config.json
Model weights saved in bigbird-roberta-base-finetuned-emrqa-msquad/checkpoint-800/model.safetensors
tokenizer config file saved in bigbird-roberta-base-finetuned-emrqa-msqu

TrainOutput(global_step=3048, training_loss=123.47734169947506, metrics={'train_runtime': 28421.9284, 'train_samples_per_second': 6.861, 'train_steps_per_second': 0.107, 'total_flos': 9.20189343912289e+16, 'train_loss': 123.47734169947506, 'epoch': 3.0})

In [21]:
# Save the model
print(f"Saving model to {new_name}...")
trainer.model.save_pretrained(new_name)
tokenizer.save_pretrained(new_name)
print("Training complete and model saved!")

Configuration saved in bigbird-roberta-base-finetuned-emrqa-msquad/config.json


Saving model to bigbird-roberta-base-finetuned-emrqa-msquad...


Model weights saved in bigbird-roberta-base-finetuned-emrqa-msquad/model.safetensors
tokenizer config file saved in bigbird-roberta-base-finetuned-emrqa-msquad/tokenizer_config.json
Special tokens file saved in bigbird-roberta-base-finetuned-emrqa-msquad/special_tokens_map.json


Training complete and model saved!


In [22]:
#loading saved model
model = AutoModelForQuestionAnswering.from_pretrained(new_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(new_name)

loading configuration file bigbird-roberta-base-finetuned-emrqa-msquad/config.json
Model config BigBirdConfig {
  "architectures": [
    "BigBirdForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attention_type": "block_sparse",
  "block_size": 64,
  "bos_token_id": 1,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 4096,
  "model_type": "big_bird",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_random_blocks": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "rescale_embeddings": false,
  "sep_token_id": 66,
  "torch_dtype": "float32",
  "transformers_version": "4.52.4",
  "type_vocab_size": 2,
  "use_bias": true,
  "use_cache": true,
  "vocab_size": 50358
}

loading weights file bigbird-roberta-base-finet

In [23]:
#getting an example from the validation data
context = ds["validation"][0]["context"]
question = ds["validation"][0]["question"]
answer = ds["validation"][0]["answers"]["text"][0]

print(f"Context:{context}\n Question:{question}\n Answer:{answer}\n ---\n")

Context:Mrs. Wetterauer is a 54-year-old female with coronary artery disease status post inferior myocardial infarction in March of 1997, with sick sinus syndrome, status post permanent pacemaker placement, and paroxysmal atrial fibrillation controlled with amiodarone; also with history of diabetes mellitus and hypertension. On 1/11, she experienced severe respiratory distress and was unable to be intubated on the field. She was ultimately intubated at Sirose, and an echocardiogram showed an ejection fraction of 25 to 30 percent with flat CKs. She was diuresed six liters and a right heart catheterization showed a pulmonary artery pressure of 40/15, wedge of 12, and cardiac output of 5.2. Hemodynamics indicated her cardiac output was dependent on her SVR. At the outside hospital, a right upper lobe infiltrate was noted and she was given gentamicin 250 mg times one, and clindamycin 600 mg. She was diagnosed with pneumonia and treated with clindamycin, which caused resolution of her white

In [24]:
#trying out the saved model
inputs = tokenizer(question, context, add_special_tokens = True, return_tensors = "pt")
inputs = {k: v.to(device) for k, v in inputs.items() if k in tokenizer.model_input_names}
input_ids = inputs["input_ids"].tolist()[0]

with torch.no_grad():
  output = model(**inputs)

start_id = torch.argmax(output.start_logits)
end_id = torch.argmax(output.end_logits) + 1

answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(input_ids[start_id:end_id])
)

print(f"Question:{question}\n Answer:{answer}\n ---\n")

Input ids are automatically padded from 1009 to 1024 to be a multiple of `config.block_size`: 64


Question:Has the patient ever taken glyburide for their diabetes mellitus:
 Answer:Glyburide 5 mg p.o. q.d.,
 ---



# `Pushing model to the hub`

In [25]:
#pushing model to my repo
repo_name = "Diaa-K/bigbird-roberta-base-finetuned-emrqa-msquad"

print(f"Pushing model to Hugging Face Hub: {repo_name}")

# Push model and tokenizer to the Hub
trainer.model.push_to_hub(repo_name, commit_message="Training Complete",token=hf_w_token)
tokenizer.push_to_hub(repo_name, commit_message="Training Complete",token=hf_w_token)

print("Model successfully pushed to Hugging Face Hub!")

Pushing model to Hugging Face Hub: Diaa-K/bigbird-roberta-base-finetuned-emrqa-msquad


README.md: 0.00B [00:00, ?B/s]

Configuration saved in bigbird-roberta-base-finetuned-emrqa-msquad/config.json
Model weights saved in bigbird-roberta-base-finetuned-emrqa-msquad/model.safetensors
Uploading the following files to Diaa-K/bigbird-roberta-base-finetuned-emrqa-msquad: README.md,model.safetensors,config.json


model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

tokenizer config file saved in bigbird-roberta-base-finetuned-emrqa-msquad/tokenizer_config.json
Special tokens file saved in bigbird-roberta-base-finetuned-emrqa-msquad/special_tokens_map.json
Uploading the following files to Diaa-K/bigbird-roberta-base-finetuned-emrqa-msquad: tokenizer.json,special_tokens_map.json,README.md,tokenizer_config.json


Model successfully pushed to Hugging Face Hub!
